In [17]:
def cross(a,b):
    return [s+t for s in a for t in b]

In [18]:
rows="ABCDEFGHI"
cols="123456789"
boxes=cross(rows,cols)
stringSudoku='..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

In [19]:
row_units = [cross(r, cols) for r in rows]

column_units = [cross(rows, c) for c in cols]

square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [20]:
def display(values):
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [21]:
def dictSudoku(boxes,sud):
    sudoku={}
    for i in range(len(sud)):
        sudoku[boxes[i]]=sud[i]
    return sudoku

In [22]:
#modified sudoku by filling missing values
def boxValues(grid):
    sudoku={}
    for i in grid:
        if(grid[i]=='.'):
            sudoku[i]='123456789'
        else:
            sudoku[i]=grid[i]
    return sudoku

In [23]:
sudoku=dictSudoku(boxes,stringSudoku)
values=boxValues(sudoku)

In [26]:
def eliminate_o(values):
    def peers(A):
        if('ABC'.find(A[0])+1):
            first='ABC'
        elif('DEF'.find(A[0])+1):
            first='DEF'
        elif('GHI'.find(A[0])+1):
            first='GHI'
        if('123'.find(A[1])+1):
            second='123'
        elif('456'.find(A[1])+1):
            second='456'
        elif('789'.find(A[1])+1):
            second='789'
        return [s+t for s in first for t in second]

    for i in values:
        if(len(values[i])!=1):
            for c in cols:
                if(len(values[i[0]+c])==1 and (i[0]+c)!=i):
                    values[i]=values[i].replace(values[i[0]+c],'')
            for r in rows:
                if(len(values[r+i[1]])==1 and (r+i[1])!=i):
                    values[i]=values[i].replace(values[r+i[1]],'')
            for cr in peers(i):
                if(len(values[cr])==1 and cr!=i):
                    values[i]=values[i].replace(values[cr],'')
    return values          

In [27]:
def eliminate(values):
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values       

In [28]:
def only_choice(values):
    for box in unitlist:
        #print(box)
        for digit in '123456789':
            collect=[]
            for cell in box:
                if digit in values[cell]:
                    collect.append(cell)
            if(len(collect)==1):
                values[collect[0]]=digit
    return values

In [29]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        sudoku=eliminate(values)
        sudoku=only_choice(sudoku)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [54]:
def reducePuzzle(values):
    values=reduce_puzzle(values)
    if values is False:
        return False
    length=0
    for b in values.keys():
        if(len(values[b])==1):
            length+=1
    if(length==81):
        return values
    else:
        n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
        temp1=values[s]
        for one in temp1:
            valuesT=values.copy()
            valuesT[s]=one
            check=reducePuzzle(valuesT)
            if check:
                return check 

In [ ]:
stringSudoku='8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..'
sudoku=dictSudoku(boxes,stringSudoku)
values=boxValues(sudoku)

In [67]:
display(reducePuzzle(values))

8 1 2 |7 5 3 |6 4 9 
9 4 3 |6 8 2 |1 7 5 
6 7 5 |4 9 1 |2 8 3 
------+------+------
1 5 4 |2 3 7 |8 9 6 
3 6 9 |8 4 5 |7 2 1 
2 8 7 |1 6 9 |5 3 4 
------+------+------
5 2 1 |9 7 4 |3 6 8 
4 3 8 |5 2 6 |9 1 7 
7 9 6 |3 1 8 |4 5 2 
